In [544]:
#from bs4 import BeautifulSoup
import re
import pandas as pd
import xml.etree.ElementTree as ET
from attribute import Attribute as att
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import *

In [175]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras import preprocessing
from keras_preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Dense, Dropout, Input, Embedding, Flatten, Dropout
from keras import Model

In [4]:
my_parser = ET.XMLParser(encoding='utf-8')
tree = ET.parse('dicom-2020a.dicomextract.txt', parser=my_parser)
#tree = ET.parse('xml DICOM', parser=my_parser)
root = tree.getroot()

In [5]:
c=[]
c = root.findall(".//ItemEntry[@Type='1C']") + root.findall(".//ItemEntry[@Type='2C']")

In [6]:
#Dictionary with conditional attributes:
attribute_dict = []
for a in c:
    description = a.attrib['Description']
    tag = a.attrib['Tag']
    type = a.attrib['Type']
    attribute_dicom = att(description,tag, type)
    attribute_dict.append(attribute_dicom)
attribute_original = deepcopy(attribute_dict)

In [7]:
for a in attribute_original:
    print(a.description, '\n')
print(len(attribute_original))

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys 

Required only if (0004,1511) is absent (see Note). 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys. 

Sequence of Items where each Item includes the Attributes of one Series to which the Presentation applies. One or more Items shall be included in this Sequence. Required if the IOD of the Presentation SOP 

In [8]:
for a in attribute_dict:
    a.set_only_rule_def()
#
# for i in attribute_dict:
#     print(i.description, '\n')

In [9]:
# Remove Notes, See Section, May be present
for a in attribute_dict:
    a.clean_note()
    a.clean_section()
    a.clean_may_be_present()
    a.clean_num_items_present()

for i in attribute_dict:
    print(i.description, '\n')


Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys 

Required only if (0004,1511) is absent (see  

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys. 

Required if the IOD of the Presentation SOP Instance referenced by this directory record includes the Presentation State Relationship Module (PS3.3:sect_C.11.11). 

Required if the SOP Instance referenced by thi

In [10]:
for a in attribute_dict:
    a.clean_enumerated_values()

for i in attribute_dict:
    print(i.description, '\n')

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys 

Required only if (0004,1511) is absent (see  

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys. 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys 

Required if an extended or replacement character set is used in one of the keys. 

Required if the IOD of the Presentation SOP Instance referenced by this directory record includes the Presentation State Relationship Module (PS3.3:sect_C.11.11). 

Required if the SOP Instance referenced by thi

In [11]:
for a in attribute_dict:
    if 'be empty' in a.description:
        print(a.description)

In [548]:
train_df = pd.read_csv('data.csv')

In [484]:
cols = train_df.columns
label_cols = list(cols[0:7])
label_cols

['Attribute description',
 'AttVal',
 'AttValNot',
 'AttPres',
 'ReqIfAttNotPres',
 'NotReqIfAttPres',
 'Txt']

In [485]:
train_df['Attribute description']

0      Required if Photometric Interpretation (0028,0...
1      Required if Intervals Acquired (0018,1083) is ...
2      Required if Line Dashing Style (0070,0254) has...
3      Required if Patient Identity Removed (0012,006...
4      Required if Respiratory Motion Compensation Te...
                             ...                        
103    Required if the Referenced SOP Instance is a S...
104    Required if the Referenced SOP Instance is a m...
105    Shall be present if Code Value (0008,0100) is ...
106    Shall be present if the code value length is 1...
107               Required only if (0004,1511) is absent
Name: Attribute description, Length: 108, dtype: object

In [549]:
def clean_train_data(df, name):
    for i in range(len(df)):
        value = re.findall('[^\\(.*][A-Z]{2,}[^(.*\\)]', df[name][i])
        tag = re.findall('\\([A-Z\\d]{4},[A-Z\\d]{4}\\)', df[name][i])
        if value:
            for val in value:
                df.loc[i,name]= df.loc[i,name].replace(val, " VALUE ")
        if tag:
            for t in tag:
                df.loc[i,name]= df.loc[i,name].replace(t, "TAG")

    return df

In [550]:
txt ='Required if Photometric Interpretation TAG has a (3476,2098) value of PALETTE COLOR or Pixel Presentation TAG at the image level equals COLOR or MIXED.'
value = re.findall('[^\\(.*][A-Z]{2,}[^(.*\\)]', txt)
tag = re.findall('\\([A-Z\\d]{4},[A-Z\\d]{4}\\)',txt)
value

[' TAG ', ' PALETTE ', 'COLOR ', ' TAG ', ' COLOR ', ' MIXED']

In [551]:
clean_train_data(train_df, 'Attribute description')
print(train_df.loc[0,'Attribute description'])
# print(cleaned_data.loc[0,'Attribute description'])

Required if Photometric Interpretation TAG has a value of VALUE  VALUE or Pixel Presentation TAG at the image level equals  VALUE or VALUE . 


In [552]:
# -----------------------------------------------MODEL TUNING AND TRAINING --------------------------------------------

In [553]:
train_df['one_hot_labels'] = list(train_df[label_cols[1:7]].values)
labels = list(train_df['one_hot_labels'].values)

In [560]:
desc = list(train_df['Attribute description'].values)
desc

['Required if Photometric Interpretation TAG has a value of VALUE  VALUE or Pixel Presentation TAG at the image level equals  VALUE or VALUE . ',
 'Required if Intervals Acquired TAG is present and has a value of 1.',
 'Required if Line Dashing Style TAG has a value of VALUE .',
 'Required if Patient Identity Removed TAG is present and has a value of VALUE and De-identification Method Code Sequence TAG is not present.',
 'Required if Respiratory Motion Compensation Technique TAG equals other than VALUE or VALUE and Respiratory Trigger Type TAG is absent or has a value of VALUE or VALUE ',
 'Shall not be present when Two Dimensional to Three Dimensional Map Sequence TAG or X Coordinates Center Pixel View Angle TAG and Y Coordinates Center Pixel View Angle TAG are present.',
 'Required if Presentation State is generated during acquisition, shall not be present otherwise. (5,6)',
 'Required if segmented data is used in an Image VALUE or Color Palette VALUE  shall not be present in a Prese

In [492]:
labels

[array([1, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 0, 1, 0, 0, 0], dtype=int64),
 array([1, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 0, 1, 1, 0, 0], dtype=int64),
 array([1, 1, 0, 1, 0, 0], dtype=int64),
 array([0, 0, 0, 0, 1, 0], dtype=int64),
 array([0, 0, 0, 0, 1, 1], dtype=int64),
 array([0, 0, 0, 0, 1, 1], dtype=int64),
 array([0, 0, 0, 1, 0, 1], dtype=int64),
 array([1, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 0, 0, 0, 0, 0], dtype=int64),
 array([0, 0, 0, 0, 0, 1], dtype=int64),
 array([0, 0, 0, 0, 0, 1], dtype=int64),
 array([0, 0, 0, 0, 0, 1], dtype=int64),
 array([0, 0, 0, 0, 0, 1], dtype=int64),
 array([1, 0, 0, 0, 0, 0], dtype=int64),
 array([1, 0, 1, 0, 0, 0], dtype=int64),
 array([0, 0, 1, 0, 0, 0], dtype=int64),
 array([0, 0, 0, 0, 0, 1], dtype=int64),
 array([0, 1, 0, 0, 0, 0], dtype=int64),
 array([0, 0, 0,

In [508]:
tok=Tokenizer(
    num_words=90000,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789',
    lower=False,
    split=' ',
    char_level=False,
    oov_token=None,
    document_count=0
)
tok.fit_on_texts(desc)
desc=tok.texts_to_sequences(desc)
desc

[[5,
  3,
  77,
  78,
  2,
  21,
  15,
  13,
  11,
  1,
  1,
  6,
  22,
  34,
  2,
  165,
  9,
  105,
  166,
  23,
  1,
  6,
  1],
 [5, 3, 167, 168, 2, 4, 7, 8, 21, 15, 13, 11],
 [5, 3, 169, 170, 171, 2, 21, 15, 13, 11, 1],
 [5,
  3,
  28,
  172,
  173,
  2,
  4,
  7,
  8,
  21,
  15,
  13,
  11,
  1,
  8,
  174,
  175,
  61,
  20,
  12,
  2,
  4,
  10,
  7],
 [5,
  3,
  55,
  79,
  80,
  56,
  2,
  23,
  29,
  18,
  1,
  6,
  1,
  8,
  55,
  106,
  14,
  2,
  4,
  30,
  6,
  21,
  15,
  13,
  11,
  1,
  6,
  1],
 [46,
  10,
  31,
  7,
  107,
  108,
  62,
  24,
  109,
  62,
  110,
  12,
  2,
  6,
  81,
  63,
  47,
  22,
  48,
  49,
  2,
  8,
  111,
  63,
  47,
  22,
  48,
  49,
  2,
  35,
  7],
 [5, 3, 34, 112, 4, 176, 50, 177, 113, 10, 31, 7, 178],
 [5,
  3,
  179,
  114,
  4,
  180,
  16,
  51,
  19,
  1,
  6,
  82,
  115,
  1,
  113,
  10,
  31,
  7,
  16,
  15,
  34,
  112,
  1],
 [5, 3, 181, 182, 4, 183, 8, 184, 185, 1, 2, 4, 10, 7],
 [5, 3, 1, 57, 2, 13, 4, 1],
 [5, 3, 34, 64, 14

In [571]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
count_vector=cv.fit_transform(desc)
count_vector.shape[1]

350

In [568]:
X_train,X_test, Y_train, Y_test= train_test_split(count_vector,labels, test_size=0.1, random_state=100)

In [557]:
# tok=Tokenizer(
#     num_words=90000,
#     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789',
#     lower=False,
#     split=' ',
#     char_level=False,
#     oov_token=None,
#     document_count=0
# )
# tok.fit_on_texts(X_train)
# X_train=tok.texts_to_sequences(X_train)
# tok.fit_on_texts(X_test)
# X_test=tok.texts_to_sequences(X_test)

In [569]:
maxlen = 500
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [572]:
X_train.shape, X_test.shape, np.asarray(Y_test).shape, np.asarray(Y_train).shape

((97, 350), (11, 350), (11, 6), (97, 6))

In [498]:
Y_test= np.asarray(Y_test)
Y_train= np.asarray(Y_train)

In [499]:
data=[]
for a in attribute_dict:
    data.append(a.description)

In [577]:
tok.fit_on_texts(data)
tok_data=tok.texts_to_sequences(data)

tok_data = pad_sequences(tok_data, padding='post', maxlen=maxlen)
tok_data

vector_data=cv.fit_transform(data)
vector_data.shape[1]

1726

In [573]:
svm = SVC(gamma = 'scale')
forest = RandomForestClassifier(random_state=40)

In [574]:
cls = ['Attribute description', 'AttVal', 'AttValNot', 'AttPres',
       'ReqIfAttNotPres', 'NotReqIfAttPres', 'Txt']
multilabel_classifier = MultiOutputClassifier(estimator=svm)
# multilabel_classifier = MultiOutputClassifier(forest, n_jobs=2)
print(multilabel_classifier)

MultiOutputClassifier(estimator=SVC())


In [575]:
multilabel_classifier = multilabel_classifier.fit(X_train, Y_train)

In [578]:
# data_pred = pad_sequences(tok_data, padding='post', maxlen=maxlen)
yhat = multilabel_classifier.predict(X_test)
# y_test_pred = multilabel_classifier.predict(vector_data)

ValueError: X has 1726 features, but SVC is expecting 350 features as input.

In [580]:
yhat[10]

array([1, 0, 0, 0, 0, 0], dtype=int64)

In [584]:
auc_y1 = roc_auc_score(Y_test[0],yhat[0])
auc_y2 = roc_auc_score(Y_test[1],yhat[1])

print("ROC AUC y1: %.4f, y2: %.4f" % (auc_y1, auc_y2))

ROC AUC y1: 0.5000, y2: 0.7500


In [545]:
sgd_linear_clf = SGDClassifier(loss='log', random_state=1, max_iter=5)
clf_w = MultiOutputClassifier(sgd_linear_clf)
clf_w.fit(X_train, Y_train)

D:\NPL_test\venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
D:\NPL_test\venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:705: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
D:\NPL_test\venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
D:\NPL_test\venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:705: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
D:\NPL_test\venv\lib\site-packages\sklearn\linear_model\_stochastic_gradient.p

MultiOutputClassifier(estimator=SGDClassifier(loss='log', max_iter=5,
                                              random_state=1))

In [532]:
for i in range(len(attribute_dict)):
    dsc = attribute_dict[i].description
    idx = y_test_pred[i]
    print('Description: ', dsc, '\n', 'Label: ', idx)

Description:  Required if an extended or replacement character set is used in one of the keys 
 Label:  [0 0 0 0 0 0]
Description:  Required if an extended or replacement character set is used in one of the keys 
 Label:  [0 0 0 0 0 0]
Description:  Required only if (0004,1511) is absent (see  
 Label:  [0 0 0 0 0 1]
Description:  Required if an extended or replacement character set is used in one of the keys 
 Label:  [0 0 0 0 0 0]
Description:  Required if an extended or replacement character set is used in one of the keys. 
 Label:  [0 0 0 0 0 0]
Description:  Required if an extended or replacement character set is used in one of the keys. 
 Label:  [0 0 0 0 0 0]
Description:  Required if an extended or replacement character set is used in one of the keys. 
 Label:  [0 0 0 0 0 0]
Description:  Required if an extended or replacement character set is used in one of the keys 
 Label:  [0 0 0 0 0 0]
Description:  Required if an extended or replacement character set is used in one of the

In [ ]:
for i in range

In [ ]:
# ------------------------MODEL 2 ---------------------------------

In [ ]:
input_1 = Input(shape=(maxlen,))
embedding_layer = Embedding(10000, 128, trainable=True)(input_1)

LSTM_Layer1 = (LSTM(128, return_sequences=True))(embedding_layer)
dropout_1 = Dropout(0.3)(LSTM_Layer1)

LSTM_Layer2 = (LSTM(128))(dropout_1)
dropout_2 = Dropout(0.3)(LSTM_Layer2)
dense_2 = Flatten()(dropout_2)

output1 = Dense(6, activation='softmax')(dense_2)  #fake news
#output2= Dense(8,activation='softmax')(dense_2)     #news subject

model = Model(inputs=input_1, outputs=output1)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()
model.fit(x=X_train, y=Y_train, batch_size=32, epochs=50)

In [ ]:
model.summary()

In [ ]:
model.fit(x=X_train,y=Y_train, batch_size=32, epochs=50)

In [ ]:
loss,acc=model.evaluate(x=X_test,y=Y_test)

In [ ]:
data = []
for a in attribute_dict:
    data.append(a.description)
tok.fit_on_texts(data)
tok_data = tok.texts_to_sequences(data)

tok_data = pad_sequences(tok_data, padding='post', maxlen=maxlen)
tok_data

In [ ]:
y_val_predicted_labels = model.predict(tok_data)
len(y_val_predicted_labels)

In [ ]:
(y_val_predicted_labels[4])

In [ ]:
for i in range(len(attribute_dict)):
    dsc = attribute_dict[i].description
    idx = np.argmax(y_val_predicted_labels[i])
    print('Description: ', dsc, '\n', 'Label: ', labels[idx])